#  WPS call Weather Regimes

The weather regimes process is divided into two parts. Based on a reference dataset (reanalyses or any kind of model dataset), the weather regimes are clustered and appropriate statisics calculated. Two processes can be used for the reference training (weatherregimes_reanalyse with a list of pre-defined reanalyses datasets, or weatherregimes_model for any suitable dataset of your choice).
The output of this first part is the forcing data for the second part. With the process weatherregimes_projection, the trained statistic can be projected onto a dataset with appropriate variables (pressure data).

In [1]:
##############################
# load the required libraries 
#############################

from owslib.wps import WebProcessingService, monitorExecution, printInputOutput

In [2]:
#################################################
# connect to the compute provider hosting the WPS
#################################################

wps_url = "http://birdhouse-lsce.extra.cea.fr:8093/wps"
# wps_url = "http://localhost:8093/wps"
wps = WebProcessingService(url=wps_url, verbose=False)

In [3]:
##########################################
# print some information about the service
##########################################

print wps.identification.title
for process in wps.processes:
    print '%s : \t %s' % (process.identifier, process.abstract)

Flyingpigeon
subset_continents : 	 Returns only the selected polygon for each input dataset
subset_countries : 	 Returns only the selected polygon for each input dataset
subset_regionseurope : 	 Returns only the selected polygon for each input dataset
subset_points : 	 Extract Timeseries for specified coordinates from gridded datasets
indices_simple : 	 Climate indices based on one single input variable.
indices_percentile : 	 Climate indices based on one single input variable and the percentile of a reference period.
weatherregimes_reanalyse : 	 Weather regimes based on pressure patterns, fetching selected reanlysis datasets
weatherregimes_model : 	 Weather regimes based on pressure patterns, fetching selected realayses datasets
weatherregimes_projection : 	 Weather Regimes detection based on trained reference statistics
analogs_detection : 	 Search for days with analog pressure pattern
analogs_viewer : 	 Visualisation of text output of analog process
segetalflora : 	 Species biodiver

In [4]:
#################################################
# print some information about a specific process
#################################################

# To recieve information, uncomment the follwing lines

#p = wps.describeprocess(identifier='weatherregimes_reanalyse')
#for input in p.dataInputs:
#  printInputOutput(input)
#  print '\n'

In [5]:
#####################
# execute the process
#####################

training = wps.execute(
    identifier="weatherregimes_reanalyse", 
    inputs=[ 
        ('season', 'DJF'),         ### uncomment the cell above 
        ('BBox','-80,22.5,50,70')  ### for more optional arguments
           ],
    async=True) 

In [6]:
##########################################################################
# the appropriate information is printed out here if the job is finished 
##########################################################################

monitorExecution(training, sleepSecs=5)
print training.getStatus()

for o in training.processOutputs:
    print o.reference

ProcessSucceeded
http://birdhouse-lsce.extra.cea.fr:8090/wpsoutputs/flyingpigeon/output_pca-2a701798-4ce0-11e6-b6cb-03216ea57d1a.dat
http://birdhouse-lsce.extra.cea.fr:8090/wpsoutputs/flyingpigeon/Routput_graphic-2a701798-4ce0-11e6-b6cb-03216ea57d1a.pdf
http://birdhouse-lsce.extra.cea.fr:8090/wpsoutputs/flyingpigeon/output_classification-2a701798-4ce0-11e6-b6cb-03216ea57d1a.Rdat
http://birdhouse-lsce.extra.cea.fr:8090/wpsoutputs/flyingpigeon/output_netcdf-2a701798-4ce0-11e6-b6cb-03216ea57d1a.nc


In [7]:
###############################################################
# Next step is to perform a projection with the trained output
###############################################################


### uncomment these lines for information:

# p = wps.describeprocess(identifier='weatherregimes_projection')
# for input in p.dataInputs:
#     printInputOutput(input)

In [8]:
#################################
# Link 
#################################
from os import listdir, getenv

HOME = getenv('HOME')

In [13]:
path = '/home/estimr2/birdhouse/birdhouse/var/lib/pywps/cache/malleefowl/esgf1.dkrz.de/thredds/fileServer/cmip5/cmip5/output1/MPI-M/MPI-ESM-LR/historical/day/atmos/day/r1i1p1/v20111006/psl/'

# can be a web url (http:// ...) as well
files = ['file://' + path + f for f in listdir(path)]
files

['file:///home/estimr2/birdhouse/birdhouse/var/lib/pywps/cache/malleefowl/esgf1.dkrz.de/thredds/fileServer/cmip5/cmip5/output1/MPI-M/MPI-ESM-LR/historical/day/atmos/day/r1i1p1/v20111006/psl/psl_day_MPI-ESM-LR_historical_r1i1p1_19800101-19891231.nc',
 'file:///home/estimr2/birdhouse/birdhouse/var/lib/pywps/cache/malleefowl/esgf1.dkrz.de/thredds/fileServer/cmip5/cmip5/output1/MPI-M/MPI-ESM-LR/historical/day/atmos/day/r1i1p1/v20111006/psl/psl_day_MPI-ESM-LR_historical_r1i1p1_19600101-19691231.nc',
 'file:///home/estimr2/birdhouse/birdhouse/var/lib/pywps/cache/malleefowl/esgf1.dkrz.de/thredds/fileServer/cmip5/cmip5/output1/MPI-M/MPI-ESM-LR/historical/day/atmos/day/r1i1p1/v20111006/psl/psl_day_MPI-ESM-LR_historical_r1i1p1_20000101-20051231.nc',
 'file:///home/estimr2/birdhouse/birdhouse/var/lib/pywps/cache/malleefowl/esgf1.dkrz.de/thredds/fileServer/cmip5/cmip5/output1/MPI-M/MPI-ESM-LR/historical/day/atmos/day/r1i1p1/v20111006/psl/psl_day_MPI-ESM-LR_historical_r1i1p1_19700101-19791231.nc',


In [14]:
########################################
# and execute the process for projection
########################################

projection = wps.execute(
    identifier="weatherregimes_projection",
    inputs=[
       ("resource",files[0]),
       ("resource",files[1]),
       ("resource",files[2]),
       ("resource",files[3]),
       ("resource",files[4]),
       ("season",'DJF'),
       ("period", "19900101-20001231"),
       ("anualcycle", "19950101-19991231"),
       ("dat", training.processOutputs[0].reference),   #  this are the output urls of the reference training
       ("Rdat", training.processOutputs[2].reference),  #
       ("netCDF",training.processOutputs[3].reference)  # the reference data are used here for regridding and bbox
           ],
    async=True)

In [15]:
monitorExecution(projection, sleepSecs=5)
print projection.getStatus()

for o in projection.processOutputs:
    print o.reference

ProcessSucceeded
http://birdhouse-lsce.extra.cea.fr:8090/wpsoutputs/flyingpigeon/output_pca-61b7d266-4ce2-11e6-b6cb-03216ea57d1a.dat
http://birdhouse-lsce.extra.cea.fr:8090/wpsoutputs/flyingpigeon/Routput_graphic-61b7d266-4ce2-11e6-b6cb-03216ea57d1a.pdf
http://birdhouse-lsce.extra.cea.fr:8090/wpsoutputs/flyingpigeon/output_classification-61b7d266-4ce2-11e6-b6cb-03216ea57d1a.Rdat
http://birdhouse-lsce.extra.cea.fr:8090/wpsoutputs/flyingpigeon/output_netcdf-61b7d266-4ce2-11e6-b6cb-03216ea57d1a.nc


In [12]:
# Voila :-) 